In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from imblearn.over_sampling import SMOTE
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

In [ ]:
#Read the dataset
df = pd.read_csv('../input/credit-card-customers/BankChurners.csv')

In [ ]:
#Checkiing data size
df.shape

In [ ]:
#Checking the top three rows of the df
df.head(3)

In [ ]:
#now checking the bottom three rows

df.tail(3)

In [ ]:
#dropping unwanted columns
df = df.drop(labels=['CLIENTNUM',
                     'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                     'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
                     axis=1)

In [ ]:
#Checking data once more
df.head()

In [ ]:
#Checking for NaN values
df.isna().sum()

In [ ]:
#Checking for Null values
df.isnull().sum()

In [ ]:
#Checking class distribution
class_counts = df.groupby('Attrition_Flag').size()
print(class_counts)

In [ ]:
print(float(class_counts['Attrited Customer']/df.shape[0])*100)

In [ ]:
print(df.skew(numeric_only=True))

In [ ]:
colors = ['#DC3220','#40B0A6']
class_counts.plot(kind='pie',ylabel='Attrition_Flag',autopct='%1.1f%%',colors=colors)

In [ ]:
#Creating the attrited customers dataframe
df_att = df[df['Attrition_Flag']=='Attrited Customer']

In [ ]:
#Creating the Existing customers dataframe
df_ext = df[df['Attrition_Flag']=='Existing Customer']

In [ ]:
df.describe()

In [ ]:
def autolabels(ax):
    rects = ax.patches
    labels = [rects[i].get_height() for i in range(len(rects))]

    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width() / 2, height + 5, label,
                ha='center', va='bottom')

In [ ]:
import matplotlib.patches as mpatches

In [ ]:
# Creating proxy artists to use in Legend 
ext_patch = mpatches.Patch(color='#40B0A6', label='Existing Customer')
att_patch = mpatches.Patch(color='#DC3220', label='Attrited Customer')

In [ ]:
#Plotting Gender distribution
ax1 = df_ext.groupby('Gender').size().plot(kind='bar',color = '#40B0A6', rot = 0)
autolabels(ax1)
ax2 = df_att.groupby('Gender').size().plot(kind='bar',color ='#DC3220', rot = 0)
autolabels(ax2)
plt.title("Gender")
plt.legend(handles=[ext_patch,att_patch],bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')

In [ ]:
df_att.groupby('Gender')['Gender'].count()/df_att.shape[0]

In [ ]:
#Plotting Education Level distribution
ax1 = df_ext.groupby('Education_Level').size().plot(kind='bar',color = '#40B0A6', width=0.9, rot = 45)
autolabels(ax1)
ax2 = df_att.groupby('Education_Level').size().plot(kind='bar',color ='#DC3220', width=0.9,rot = 45)
autolabels(ax2)
plt.title("Education Level")
plt.legend(handles=[ext_patch,att_patch],bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')

In [ ]:
edu_PercAttr = (df_att.groupby('Education_Level').size()/df_att.shape[0])*100
edu_PercAttr

In [ ]:
#Plotting Marital_Status distribution
ax1 = df_ext.groupby('Marital_Status').size().plot(kind='bar',color = '#40B0A6', width=0.9, rot = 0)
autolabels(ax1)
ax2 = df_att.groupby('Marital_Status').size().plot(kind='bar',color ='#DC3220', width=0.9,rot = 0)
autolabels(ax2)
plt.title("Marital Status")
plt.legend(handles=[ext_patch,att_patch],bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')

In [ ]:
#Plotting Income Category distribution
ax1 = df_ext.groupby('Income_Category').size().plot(kind='bar',color = '#40B0A6', width=0.9, rot = 45)
autolabels(ax1)
ax2 = df_att.groupby('Income_Category').size().plot(kind='bar',color ='#DC3220', width=0.9,rot = 45)
autolabels(ax2)
plt.title("Income Category")
plt.legend(handles=[ext_patch,att_patch],bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')

In [ ]:
#Which income groups show the highest level of attrition?
income_attr = (df_att.groupby('Income_Category').size()/df_att.shape[0])*100
print("Attrited Customers - Income Category",income_attr)

In [ ]:
#Percentage distribution of Income Category in the original dataset
print((df.groupby('Income_Category').size()/df.shape[0])*100)

In [ ]:
#Plotting Card Category distribution
ax1 = df_ext.groupby('Card_Category').size().plot(kind='bar',color = '#40B0A6', width=0.9, rot = 0)
autolabels(ax1)
ax2 = df_att.groupby('Card_Category').size().plot(kind='bar',color ='#DC3220', width=0.9,rot = 0)
autolabels(ax2)
plt.title("Card Category")
plt.legend(handles=[ext_patch,att_patch],bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')

In [ ]:
# What are the card categories of Attrited Customers
card_attr = (df_att.groupby('Card_Category').size()/df_att.shape[0])*100
print(card_attr)

In [ ]:
#How much percentage of a certain card category has left?
df_att.groupby('Card_Category').size()/df.groupby('Card_Category').size()

In [ ]:
#Plotting Number of months inactive distribution
ax1 = df_ext.groupby('Months_Inactive_12_mon').size().plot(kind='bar',color = '#40B0A6', width=0.9, rot = 0)
autolabels(ax1)
ax2 = df_att.groupby('Months_Inactive_12_mon').size().plot(kind='bar',color ='#DC3220', width=0.9,rot = 0)
autolabels(ax2)
plt.title("Months Inactive")
plt.legend(handles=[ext_patch,att_patch],bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')

In [ ]:
#Median months inactive by existing and attrited customers
df.groupby('Attrition_Flag')['Months_Inactive_12_mon'].median()

In [ ]:
df.skew()

**Numerical Variables Analysis:**

In [ ]:
(df.groupby('Card_Category')['Total_Trans_Amt'].sum()/df['Total_Trans_Amt'].sum())*100

In [ ]:
df.groupby('Card_Category')['Total_Trans_Amt'].sum(),df['Total_Trans_Amt'].sum()

In [ ]:
df.groupby('Attrition_Flag')['Months_Inactive_12_mon'].median()

In [ ]:
#Plotting Density plots
fig, ax = plt.subplots(figsize=(14,10))
df.plot(kind='density',subplots = True, layout=(5,3),sharex=False, ax = ax)

In [ ]:
df.cov()

In [ ]:
#Age distribution
c={'Existing Customer':'#40B0A6','Attrited Customer':'#DC3220'}
df.groupby('Attrition_Flag')['Customer_Age'].plot.hist(color=c, alpha=0.5)
plt.title("Customer Age - Histogram")
plt.legend()

In [ ]:
#Months on book
c={'Existing Customer':'#40B0A6','Attrited Customer':'#DC3220'}
df.groupby('Attrition_Flag')['Months_on_book'].plot.hist(color=c, alpha=0.5)
plt.title('Months on Book - Histogram')
plt.legend()

In [ ]:
#Creating a df of transaction variables
df_trans = df[['Credit_Limit','Total_Revolving_Bal','Avg_Open_To_Buy','Total_Amt_Chng_Q4_Q1','Total_Trans_Amt','Total_Trans_Ct','Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio','Attrition_Flag']]

In [ ]:
df_trans = df_trans.replace({'Attrited Customer':1,'Existing Customer':0})

In [ ]:
correlations = df_trans.corr()

In [ ]:
#plotting heatmap
ax = sns.heatmap(
    correlations,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    annot=True,
    fmt='.1g'
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation = 45,
horizontalalignment = 'right'
);

In [ ]:
df_selected = df[['Total_Revolving_Bal','Total_Trans_Ct','Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio','Attrition_Flag']]

In [ ]:
sns.pairplot(df_selected, hue='Attrition_Flag',palette = {'Existing Customer':'#40B0A6','Attrited Customer':'#DC3220'})

In [ ]:
#Box plot of Total Transactions
sns.boxplot(x=df['Attrition_Flag'],y=df['Total_Trans_Ct'],
            hue=df['Attrition_Flag'], palette={'Existing Customer':'#40B0A6','Attrited Customer':'#DC3220'}
            )
sns.despine(offset=10, trim=True)

In [ ]:
df.groupby('Attrition_Flag')['Total_Trans_Ct'].describe()

In [ ]:
#Box plot of Total_Ct_Chng_Q4_Q1
sns.boxplot(x=df['Attrition_Flag'],y=df['Total_Ct_Chng_Q4_Q1'],
            hue=df['Attrition_Flag'], palette={'Existing Customer':'#40B0A6','Attrited Customer':'#DC3220'}
            )
sns.despine(offset=10, trim=True)

In [ ]:
df.groupby('Attrition_Flag')['Total_Ct_Chng_Q4_Q1'].describe()

In [ ]:
df[df['Total_Ct_Chng_Q4_Q1']==0]['Attrition_Flag']

In [ ]:
#Box plot of Avg_Utilization_Ratio
sns.boxplot(x=df['Attrition_Flag'],y=df['Avg_Utilization_Ratio'],
            hue=df['Attrition_Flag'], palette={'Existing Customer':'#40B0A6','Attrited Customer':'#DC3220'}
            )
sns.despine(offset=10, trim=True)

In [ ]:
df.groupby('Attrition_Flag')['Avg_Utilization_Ratio'].describe()

In [ ]:
df.groupby('Attrition_Flag')['Avg_Utilization_Ratio'].median()

In [ ]:
df_att['Avg_Utilization_Ratio'].mean()

In [ ]:
#Box plot of Total_Revolving_Bal
sns.boxplot(x=df['Attrition_Flag'],y=df['Total_Revolving_Bal'],
            hue=df['Attrition_Flag'], palette={'Existing Customer':'#40B0A6','Attrited Customer':'#DC3220'}
            )
sns.despine(offset=10, trim=True)

In [ ]:
df.groupby('Attrition_Flag')['Total_Revolving_Bal'].describe()

In [ ]:
df.groupby('Attrition_Flag')['Total_Revolving_Bal'].median()

In [ ]:
df_chosen = df[['Total_Trans_Ct', 'Total_Revolving_Bal',
                'Avg_Utilization_Ratio', 'Attrition_Flag']]

In [ ]:
# print description of selected features
print("Description of selected features: \n", df_chosen.describe())

# print description of each selected variable split by Target
print("Total_Trans_Ct : \n", df.groupby(
    'Attrition_Flag')['Total_Trans_Ct'].describe(), "\n")
print("Total_Revolving_Bal: \n", df.groupby(
    'Attrition_Flag')['Total_Revolving_Bal'].describe(), "\n")
print("Avg_Utilization_Ratio: \n", df.groupby(
    'Attrition_Flag')['Avg_Utilization_Ratio'].describe(), "\n")

In [ ]:
#Visualising the chosen dimensions - Scatterplot with Target shaded by colour

sns.pairplot(df_chosen, hue='Attrition_Flag', diag_kind="hist", palette={
    'Existing Customer': '#40B0A6', 'Attrited Customer': '#DC3220'})
plt.show()

In [ ]:
# creating a list of chosen dimensions
xl = list(df_chosen['Total_Trans_Ct'])
yl = list(df_chosen['Total_Revolving_Bal'])
zl = list(df_chosen['Avg_Utilization_Ratio'])
data_points = [(x, y, z) for x, y, z in zip(xl, yl, zl)]

In [ ]:
# setting Target colours
colors = ['#40B0A6' if flag == 'Existing Customer' else '#DC3220' for flag in list(
    df_chosen['Attrition_Flag'])]

In [ ]:
def plotter(data_points, title, xlabel, ylabel, zlabel):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    for data, color in zip(data_points, colors):
        x, y, z = data
        ax.scatter(x, y, z, alpha=0.9, c=color, edgecolors='white', s=30)

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_zlabel(zlabel)
    ax.set_title(title)
    plt.show()


def plotCumExplVar(ve, title):
    plt.plot(ve.cumsum())
    plt.title(title)
    plt.xlabel('Principal Component')
    plt.xticks(np.arange(0, 3, step=1), ['PC1', 'PC2', 'PC3'])
    plt.ylabel('Cumulative Explained Variance')
    plt.show()


plotter(data_points, '3D Plot of chosen Dimensions – Target shaded by colour', 'Total_Trans_Ct',
        'Total_Revolving_Bal', 'Avg_Utilization_Ratio')

In [ ]:
# PCA on scaled data
# Normalise dimensions to a zero mean and unit SD
scaled_data = scale(data_points)  # Scales and centers the data

# Do PCA on scaled data
pca_scaled = PCA(n_components=3)
scaled_pc_data = pca_scaled.fit_transform(scaled_data)
print("Scenario 1: PCA on scaled data. Constructing PC Outputs...")
plotter(scaled_pc_data, 'PCA on scaled data - Scatter Plot', '1st eigenvector',
        '2nd eigenvector', '3rd eigenvector')

# Explained variance
# Gives proportion of variance explained for each principal component
var_explained = pca_scaled.explained_variance_ratio_
print("PCA on scaled data - Explained Variance :", var_explained)
print("Constructing Cumulative Explained Variance plot...")
plotCumExplVar(
    var_explained, 'PCA on scaled data - Cumulative Explained Variance')

# printing PCA components breakdown
print("PCA Components (Scaled data) : ", pca_scaled.components_)

In [ ]:
# Do PCA on unscaled data
pca_unscaled = PCA(n_components=3)
unscaled_pc_data = pca_unscaled.fit_transform(data_points)
print("Scenario 2: PCA without scaling. Constructing PC Outputs...")
plotter(unscaled_pc_data, 'PCA on original data (without scaling) - Scatter Plot', '1st eigenvector',
        '2nd eigenvector', '3rd eigenvector')

# Explained variance
# Gives proportion of variance explained for each principal component
var_explained = pca_unscaled.explained_variance_ratio_
print("PCA on original data (without scaling) - Explained Variance :", var_explained)
print("Constructing Cumulative Explained Variance plot...")
plotCumExplVar(
    var_explained, 'PCA on original data (without scaling) - Cumulative Explained Variance')
# printing PCA components breakdown
print("PCA Components (Original data) : ", pca_unscaled.components_)

In [ ]:
# using SMOTE to address class imbalance

X = df_chosen.iloc[:, :-1]
Y = df_chosen.iloc[:, -1:]

sm = SMOTE(sampling_strategy='auto', random_state=1234)
x_sm, y_sm = sm.fit_resample(X, Y)

# Checking class counts
class_counts = y_sm.groupby('Attrition_Flag').size()
print(class_counts)

# Plotting class distribution
colors = ['#DC3220', '#40B0A6']
class_counts.plot(kind='pie', ylabel='Attrition_Flag',
                  autopct='%1.1f%%', colors=colors)
plt.show()

# constructing a list of resampled datapoints
xl = list(x_sm['Total_Trans_Ct'])
yl = list(x_sm['Total_Revolving_Bal'])
zl = list(x_sm['Avg_Utilization_Ratio'])
data_points = [(x, y, z) for x, y, z in zip(xl, yl, zl)]

# Visualising the resampled data
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Do PCA on re-sampled balanced data
pca_balanced = PCA(n_components=3)
balanced_pc_data = pca_balanced.fit_transform(data_points)

print("Scenario 3: PCA on balanced data (after SMOTE). Constructing PC Outputs...")

colors = ['#40B0A6' if flag == 'Existing Customer' else '#DC3220' for flag in list(
    y_sm['Attrition_Flag'])]
for data, color in zip(balanced_pc_data, colors):
    x, y, z = data
    ax.scatter(x, y, z, alpha=0.9, c=color, edgecolors='white', s=30)

ax.set_title('PCA on balanced data (after SMOTE) - Scatter Plot')
ax.set_xlabel('Total_Trans_Ct')
ax.set_ylabel('Total_Revolving_Bal')
ax.set_zlabel('Avg_Utilization_Ratio')
plt.show()

# Explained variance
# Gives proportion of variance explained by each principal component
var_explained_balanced = pca_balanced.explained_variance_ratio_
print("PCA on balanced data (after SMOTE) - Explained Variance :",
      var_explained_balanced)
print("Constructing Cumulative Explained Variance plot...")
plotCumExplVar(
    var_explained_balanced, 'PCA on balanced data (after SMOTE) - Cumulative Explained Variance')

# printing PCA components breakdown
print("PCA Components (Balanced data) : ", pca_balanced.components_)